# Can I walk to the train?
In this notebook we'll be exploring the walkability of Klang Valley's train stations using isochrone maps
We will use Open Route Service's API to draw this and compile the results into a csv
The data gathered will be visualized to be understood
KL has multiple different train lines with varying levels of service spreading across a large area. There is a relatively large number of stations relative to the population however the accesibility of the stations seems to be an issue for pedestrians. We'll analyze each train station from different lines. And compile them later on to get an average to be compared with other cities

The work from this notebook was presented in a Twitter thread at the following link:- <br>
https://twitter.com/NabilErsyad/status/1325415849809727489?s=20

In [1]:
#folium is the library used to visualize maps
import folium
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import pandas as pd 
import numpy as np 
import plotly as py
import os
import time

from datetime import date
from isochrones import * 

#i like to be able to see all my columns in a pandas dataframe. this would allow it 
pd.set_option("display.max_columns", None)


# Train Station isochrones

Let's visualize all the stations and the 15 minute walking radius on a map.
15 minutes is somewhat arbitrary but given malaysia's hot weather it seems the most fair amount of time that malaysian's can generally tolerate to walk outdoors

In [2]:
#Personal api_key stored in a config.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key

from config import api_key

clnt = client.Client(key=api_key) 

In [3]:
#loading up train station data. This file contains the csv data for KL train stations. 
#using the os path joiner to ensure that the paths will work across platforms
#essentially calling the following 'resources/data/raw_data/klang_valley_stations.csv'

file_kl = 'klang_valley_stations.csv'
raw_data_folder = 'raw_data'
data_folder = 'data'
resources_folder = 'resources'

file_url =  file_path = os.path.join(resources_folder,data_folder,raw_data_folder, file_kl)

#Stop ID is unique for every station. So that column will be used for index
data = pd.read_csv(file_url)


# We would now normalize/standardize the column names of the dataframe to ensure consistency
data.columns = data.columns.str.lower().str.replace(' ', '_')
print(data.columns)
data.head()



Index(['name', 'stop_id', 'service_provider_name', 'latitude', 'longitude',
       'route_id', 'route_name', 'line_number', 'line_colour',
       'colour_hex_code', 'city'],
      dtype='object')


,name,stop_id,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,city
0,KL SENTRAL,KA01,Keretapi Tanah Melayu,3.134603,101.686567,KA,Seremban Line,1,Blue,#0000FF,Klang Valley
1,KUALA LUMPUR,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley
2,BANK NEGARA,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley
3,PUTRA,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley
4,MID VALLEY,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley


# isochrone.py package and ORS client to get isochrones
We'll be using the function toMap() that was imported from the custom isochrone.py package. 
All you need for the function are:<br>
-- dataframe with the columns - ['Name','Route Name','Latitude','Longitude'] that would be sufficient to draw isochrone maps for train stations.<br>
-- param_iso - dictionary with the parameters of the isochrone <br>
-- client - the open route service api client <br>

Will return isochrone map of a transit line and dictionary of stations

In [4]:
#the way I coded this is that I want maps that will display train stations by lines. So that it wouldnt clutter the map.

# Gather unique train lines
train_lines = list(data['route_name'].unique())

# Initialize empty list to hold map objects
map_objects = []

isochrone_params = {
              'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
# Iterate through each train line to create a map object

#TODO I should edit the toMap function, if params_iso or client is not passed then just map without calling isochrones from ORS           
for line in train_lines:
    try:
      map_objects.append(toMapORS(data, line, isochrone_params, clnt))
      time.sleep(1) #delay added to avoid rate limit due to account limit of using the ORS API
    except Exception as e:
      print(f"Failed to generate map for {line} due to {e}")

/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["longitude"],row.loc["latitude"]]) , axis = 1)


Retrieving Isochrone of KL SENTRAL station
Failed to retrieve isochrone for station KL SENTRAL: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2023-06-16T03:06:42Z', 'version': '7.1.0'}, 'timestamp': 1688203494769}})
Retrieving Isochrone of KUALA LUMPUR station
Success
Retrieving Isochrone of BANK NEGARA station
Success
Retrieving Isochrone of PUTRA station
Success
Retrieving Isochrone of MID VALLEY station
Success
Retrieving Isochrone of SEPUTEH station
Success
Retrieving Isochrone of SALAK SELATAN station
Success
Retrieving Isochrone of BANDAR TASIK SELATAN station
Success
Retrieving Isochrone of SERDANG station
Success
Retrieving Isochrone of KAJANG station
Success
Retrieving Isochrone of UKM station
Success
Retrieving Isochrone of BANGI station
Success
Retrieving Isochrone of BATANG BENAR station
Success
Retrieving Isochrone of NILAI station
Failed to retrieve isochrone for station NILAI: 500 ({'error': {'code': 309

/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.fo

Success
Retrieving Isochrone of SENTUL station
Success
Retrieving Isochrone of BATU KENTOMEN station
Success
Retrieving Isochrone of KAMPUNG BATU station
Success
Retrieving Isochrone of TAMAN WAHYU station
Success
Retrieving Isochrone of BATU CAVES station
Failed to retrieve isochrone for station BATU CAVES: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2023-06-16T03:06:42Z', 'version': '7.1.0'}, 'timestamp': 1688203586079}})
Failed to generate map for Seremban Line due to Cannot render objects with any missing geometries: None
Retrieving Isochrone of KL SENTRAL station


/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["longitude"],row.loc["latitude"]]) , axis = 1)
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:130: SettingWithCopyWarning:

...

/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["longitude"],row.loc["latitude"]]) , axis = 1)
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:130: SettingWithCopyWarning:

Success
Retrieving Isochrone of KAYU ARA station


/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.fo

Success
Retrieving Isochrone of BU11 station
Success
Retrieving Isochrone of DAMANSARA IDAMAN station
Success
Retrieving Isochrone of SS7 station
Success
Retrieving Isochrone of GLENMARIE station
Success
Retrieving Isochrone of KERJAYA station
Success
Retrieving Isochrone of STADIUM SHAH ALAM station
Failed to retrieve isochrone for station STADIUM SHAH ALAM: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2023-06-16T03:06:42Z', 'version': '7.1.0'}, 'timestamp': 1688204626697}})
Retrieving Isochrone of BANDARAYA SHAH ALAM station
Success
Retrieving Isochrone of UITM station
Success
Retrieving Isochrone of HOSPITAL SHAH ALAM station
Success
Retrieving Isochrone of BANDAR BARU KLANG station
Success
Retrieving Isochrone of PASAR BESAR KLANG station
Success
Retrieving Isochrone of JALAN MERU station
Success
Retrieving Isochrone of KLANG station
Success
Retrieving Isochrone of TAMAN SELATAN station
Success
Retrieving Isochron

/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["longitude"],row.loc["latitude"]]) , axis = 1)


# Saving isochrone data into CSVs
Once all the Isochrone requests are completed, I would want to store the data first into CSVs for future references. <br>
The following helper functions from isochrones.py will do that<br>
dictToDataFrame(maps,dataframe)
areaToDataframe(data)

In [5]:
map_objects[0][1].columns

Index(['name', 'stop_id', 'service_provider_name', 'latitude', 'longitude',
       'route_id', 'route_name', 'line_number', 'line_colour',
       'colour_hex_code', 'city', 'locations', 'iso'],
      dtype='object')

In [6]:
#saves the data with the isochrones  data into new csv files
data= pd.DataFrame(columns=map_objects[0][1].columns)

for mapz in map_objects:
    data = pd.concat([data,mapz[1]],ignore_index=True)
file_url ='resources/data/klang_valley_stations_isochrones_' + str(date.today())+ '.csv'
data.to_csv(file_url,index=False)

In [7]:
import json
temp_path = '/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/resources/data/klang_valley_stations_isochrones_2021-11-19.csv'
temp_data = pd.read_csv(temp_path)

isonew = data.loc[0]['iso']
isotemp = temp_data.loc[0]['iso']

#print((isonew))
evale = json.loads(isonew)
area1 =  evale['features'][0]['properties']['area']/1000000
file_url ='/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/resources/data/klang_valley_stations_isochrones_2022-03-03.csv'


FileNotFoundError: [Errno 2] No such file or directory: '/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/resources/data/klang_valley_stations_isochrones_2021-11-19.csv'

In [8]:
#used to extract more specific data from isochrone data.
import isochrones as isos
data = pd.read_csv(file_url)
data  = isos.areaToDataframe(data)
data.to_csv(file_url,index=False)

In [9]:
map_objects[0][1]

,name,stop_id,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,city,locations,iso
144,KL SENTRAL,KE1,Express Rail Link,3.132675,101.687891,KE,KLIA Ekspres,6,Purple,#800080,Klang Valley,"[101.687891, 3.132675]","{""type"": ""FeatureCollection"", ""metadata"": {""at..."
145,KUALA LUMPUR INTERNATIONAL AIRPORT,KE2,Express Rail Link,2.754926,101.704825,KE,KLIA Ekspres,6,Purple,#800080,Klang Valley,"[101.7048248, 2.7549259]","{""type"": ""FeatureCollection"", ""metadata"": {""at..."
146,KLIA2,KE3,Express Rail Link,2.744569,101.685081,KE,KLIA Ekspres,6,Purple,#800080,Klang Valley,"[101.685081, 2.7445694]","{""type"": ""FeatureCollection"", ""metadata"": {""at..."


In [10]:
map_objects[0][0]

In [21]:
map_objects[1][0]

In [11]:
map_objects[2][0]

In [22]:
map_objects[3][0]

In [13]:
map_objects[4][0]

In [14]:
map_objects[5][0]

In [15]:
map_objects[6][0]

In [16]:
map_objects[7][0]

In [17]:
map_objects[8][0]

In [18]:
map_objects[9][0]

In [19]:
map_objects[10][0]

In [23]:
map_objects[11][0]

# Kuching
Trying out for provisional Kuching

In [5]:
#loading up train station data.
file_url = 'resources/data/raw_data/KUTS_Provisional_Stations.csv'

data_kuch = pd.read_csv(file_url)
lines = list(data_kuch['Route Name'].unique())
map_kuch = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines:
    map_kuch.append(toMapORS(data_kuch,line,params_iso,clnt))

Retrieving Isochrone of UNIMAS Pintu Timur station
Success
Retrieving Isochrone of Aiman Mall station
Success
Retrieving Isochrone of Wisma SALCRA station
Success
Retrieving Isochrone of UNIMAS station
Success
Retrieving Isochrone of Sarawak Heart Centre station
Success
Retrieving Isochrone of Riveria station
Success
Retrieving Isochrone of NorthBank station
Success
Retrieving Isochrone of Tabuan Jaya Police Complex station
Success
Retrieving Isochrone of VivaCity Megamall station
Success
Retrieving Isochrone of Wisma Persekutuan Public Carpark station
Success
Retrieving Isochrone of The Spring station
Success
Retrieving Isochrone of Kuching Public Works Department station
Success
Retrieving Isochrone of Sarawak General Hospital station
Success
Retrieving Isochrone of Padang Pasir, Hikmah Exchange station
Success
Done!
Retrieving Isochrone of JPJ station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of 10th Mile Bazaar station
Success
Retrieving Isochrone of SMK Wira Penrissen station
Success
Retrieving Isochrone of Sentosa Parade station
Success
Retrieving Isochrone of Kuching Sentral station
Success
Retrieving Isochrone of Kuching International Airport station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of RH Plaza station
Success
Retrieving Isochrone of Emporium station
Success
Retrieving Isochrone of Swinburne University station
Success
Retrieving Isochrone of Wisma Persekutuan Public Carpark station
Success
Retrieving Isochrone of Chung Hua Middle School No. 3 station
Success
Retrieving Isochrone of Puspakom station
Success
Retrieving Isochrone of Bintawa Industrial Estate station
Success
Retrieving Isochrone of Borneo Convention Centre Kuching station
Success
Done!


In [10]:
#saves the data with the isochrones  data into new csv files
data_kuch =map_kuch[0][1].append(map_kuch[1][1])
file_url ='resources/data/kuching_provisional_stations_isochrones_' + str(date.today())+ '.csv'
data_kuch.to_csv(file_url,index=False)

In [11]:
#used to extract more specific data from isochrone data.
data_kuch = pd.read_csv(file_url)
data_kuch  = areaToDataframe(data_kuch)
data_kuch.to_csv(file_url,index=False)

In [7]:
map_kuch[0][0]

In [9]:
map_kuch[1][0]

# Singapore
Will be doing the same but with Singapore train station data

In [4]:
#loading up train station data.
file_url = 'resources/data/raw_data/mrtsg.csv'
data_sg = pd.read_csv(file_url)
lines = list(data_sg['Route Name'].unique())
map_sg = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines:
    map_sg.append(toMapORS(data_sg,line,params_iso,clnt))

/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of Eunos MRT Station station
Success
Retrieving Isochrone of Chinese Garden MRT Station station
Success
Retrieving Isochrone of Redhill MRT Station station
Success
Retrieving Isochrone of Pioneer MRT Station station
Success
Retrieving Isochrone of Commonwealth MRT Station station
Success
Retrieving Isochrone of Paya Lebar MRT Station station
Success
Retrieving Isochrone of Simei MRT Station station
Success
Retrieving Isochrone of Tampines MRT Station station
Success
Retrieving Isochrone of Pasir Ris MRT Station station
Success
Retrieving Isochrone of Clementi MRT Station station
Success
Retrieving Isochrone of Aljunied MRT Station station
Success
Retrieving Isochrone of Jurong East MRT Station station
Success
Retrieving Isochrone of Queenstown MRT Station station
Success
Retrieving Isochrone of Tuas Link MRT Station station
Success
Retrieving Isochrone of Gul Circle MRT Station station
Success
Retrieving Isochrone of Tuas Crescent MRT Station station
Success
Retrie

/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.fo

Success
Retrieving Isochrone of Boon Lay MRT Station station
Success
Retrieving Isochrone of Bedok MRT Station station
Success
Retrieving Isochrone of Joo Koon MRT Station station
Success
Retrieving Isochrone of Lakeside MRT Station station
Success
Retrieving Isochrone of Tanah Merah MRT Station station
Success
Retrieving Isochrone of City Hall MRT Station station
Success
Retrieving Isochrone of Tanjong Pagar MRT Station station
Success
Retrieving Isochrone of Bugis MRT Station station
Success
Retrieving Isochrone of Tiong Bahru MRT Station station
Success
Retrieving Isochrone of Lavender MRT Station station
Success
Retrieving Isochrone of Raffles Place MRT Station station
Success
Retrieving Isochrone of Outram Park MRT Station station
Success
Done!
Retrieving Isochrone of Khatib MRT Station station


/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:126: SettingWithCopyWarning:

...

/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:126: SettingWithCopyWarning:

Success
Retrieving Isochrone of Woodlands MRT Station station
Success
Retrieving Isochrone of Woodlands MRT North Station station
Success
Retrieving Isochrone of Springleaf MRT Station station
Success
Retrieving Isochrone of Lentor MRT Station station
Success
Retrieving Isochrone of Mayflower MRT Station station
Success
Retrieving Isochrone of Bright Hill MRT Station station
Success
Retrieving Isochrone of Upper Thomson MRT Station station


/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/nabilersyad/miniconda3/envs/ors/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.fo

Success
Retrieving Isochrone of Caldecott MRT Station station
Success
Retrieving Isochrone of Stevens MRT Station station
Success
Retrieving Isochrone of Napier MRT Station station
Success
Retrieving Isochrone of Orchard Boulevard MRT Station station
Success
Retrieving Isochrone of Orchard MRT Station station
Success
Retrieving Isochrone of Great World MRT Station station
Success
Retrieving Isochrone of Havelock MRT Station station
Success
Retrieving Isochrone of Outram Park MRT Station station
Success
Retrieving Isochrone of Maxwell MRT Station station
Success
Retrieving Isochrone of Shenton Way MRT Station station
Success
Retrieving Isochrone of Marina Bay MRT Station station
Success
Retrieving Isochrone of Gardens By The Bay • Taman Di Pesisiran MRT Station station
Success
Done!


/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/Users/nabilersyad/Public/local_mac_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


In [5]:
#saves the data with the isochrones  data into new csv files
data_temp = pd.DataFrame(columns=map_sg[0][1].columns)
for mapz in map_sg:
    data_temp = pd.concat([data_temp,mapz[1]],ignore_index=True)
file_url ='resources/data/mrtsg_isochrones_' + str(date.today())+ '.csv'
data_temp.to_csv(file_url,index=False)

In [6]:
#used to extract more specific data from isochrone data.
data_sg = pd.read_csv(file_url)
data_sg  = areaToDataframe(data_sg)
data_sg.to_csv(file_url,index=False)

In [7]:
map_sg[0][0]

In [8]:
map_sg[1][0]

In [9]:
map_sg[2][0]

In [10]:
map_sg[3][0]

In [11]:
map_sg[4][0]

In [12]:
map_sg[5][0]

In [13]:
map_sg[6][0]

In [15]:
map_sg[7][0]

In [16]:
map_sg[8][0]

In [17]:
map_sg[9][0]

In [18]:
map_sg[10][0]

# Montreal
Again same methodology but for Montreal

In [29]:
#loading up train station data.
file_name_montreal = 'resources/data/raw_data/montreal_metro.csv'
data_montreal = pd.read_csv(file_name_montreal)
lines_montreal = list(data_montreal['Route Name'].unique())
maps_montreal = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
for line in lines_montreal:
    maps_montreal.append(toMapORS(data_montreal,line,params_iso,clnt))



/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of Angrignon station
Success
Retrieving Isochrone of Monk station
Success
Retrieving Isochrone of Jolicoeur station
Success
Retrieving Isochrone of Verdun station
Success
Retrieving Isochrone of De L'Église station
Success
Retrieving Isochrone of LaSalle station
Success
Retrieving Isochrone of Charlevoix station
Success
Retrieving Isochrone of Lionel-Groulx station
Success
Retrieving Isochrone of Atwater station
Success
Retrieving Isochrone of Guy–Concordia station
Success
Retrieving Isochrone of Peel station
Success
Retrieving Isochrone of McGill station
Success
Retrieving Isochrone of Place-des-Arts station
Success
Retrieving Isochrone of Saint-Laurent station
Success
Retrieving Isochrone of Berri–UQAM station
Success
Retrieving Isochrone of Beaudry station
Success
Retrieving Isochrone of Papineau station
Success
Retrieving Isochrone of Frontenac station
Success
Retrieving Isochrone of Préfontaine station
Success
Retrieving Isochrone of Joliette station
Success
R

/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(ret

Success
Retrieving Isochrone of Viau station
Success
Retrieving Isochrone of Assomption station
Success
Retrieving Isochrone of Cadillac station
Success
Retrieving Isochrone of Langelier station
Success
Retrieving Isochrone of Radisson station
Success
Retrieving Isochrone of Honoré-Beaugrand station
Success
Done!
Retrieving Isochrone of Côte-Vertu station


/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list
/home/nabilersyad/Desktop/LinuxDrive/local_repo/train-stations-isochrones/isochrones.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Success
Retrieving Isochrone of Du Collège station
Success
Retrieving Isochrone of De La Savane station
Success
Retrieving Isochrone of Namur station
Success
Retrieving Isochrone of Plamondon station
Success
Retrieving Isochrone of Côte-Sainte-Catherine station
Success
Retrieving Isochrone of Snowdon station
Success
Retrieving Isochrone of Villa-Maria station
Success
Retrieving Isochrone of Vendôme station
Success
Retrieving Isochrone of Place-Saint-Henri station
Success
Retrieving Isochrone of Lionel-Groulx station
Success
Retrieving Isochrone of Georges-Vanier station
Success
Retrieving Isochrone of Lucien-L'Allier station
Success
Retrieving Isochrone of Bonaventure station
Success
Retrieving Isochrone of Square-Victoria–OACI station
Success
Retrieving Isochrone of Place-d'Armes station
Success
Retrieving Isochrone of Champ-de-Mars station
Success
Retrieving Isochrone of Berri–UQAM station
Success
Retrieving Isochrone of Sherbrooke station
Success
Retrieving Isochrone of Mont-Royal s

In [32]:
#saves the data with the isochrones  data into new csv files
data_temp = pd.DataFrame(columns=maps_montreal[0][1].columns)
for mapz in maps_montreal:
    data_temp = data_temp.append(mapz[1])
file_name_montreal ='resources/data/montreal_metro_isochrones_' + str(date.today())+ '.csv'
data_temp.to_csv(file_name_montreal,index=False)

In [34]:
#used to extract more specific data from isochrone data.
data_montreal = pd.read_csv(file_name_montreal)
data_montreal  = areaToDataframe(data_montreal)
data_montreal.to_csv(file_name_montreal,index=False)

In [35]:
maps_montreal[0][0]

In [260]:
maps_montreal[1][0]

In [261]:
maps_montreal[2][0]

In [262]:
maps_montreal[3][0]

# We've got isochrones!

We're done gathering data. Now to get some stats we'll be doing it in a different notebook as it's getting too long here